In [1]:
!pip install deepface  

  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cache

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 ade03804d81e696f8b9045bbe2dd5d0146e36c63d85bf2eae8225ffa74a03713
             Got        cde1488fd4f567e33c5241958d1ece5a8239b99d4d72783f82c12a3c84683c9c



In [17]:
%%writefile app.py
import streamlit as st
from deepface import DeepFace
from deepfacert  import DeepFace
import cv2
import numpy as np
from PIL import Image
import tempfile




def Analyze_Emotion(img_path):
    try:
        analysis = DeepFace.analyze(
        img_path,
        actions=['emotion'],
        enforce_detection=False)
        return analysis[0]['emotion']
    except Exception as e:
        st.error(f"Error analyzing image: {e}")
        return None



options = st.selectbox('Select an option',
    ('Upload Image', 'Use Webcam'))


if options == 'Image':
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        img_array = np.array(image)
        st.image(img_array, caption='Uploaded Image.', use_column_width=True)
        emotion_scores = Analyze_Emotion(uploaded_file)
        if emotion_scores:
            detected_emotion = max(emotion_scores, key=emotion_scores.get)
            st.write(f"Detected Emotion: {detected_emotion}")
        else:   
            st.write("No emotion detected or an error occurred.")
            
            
if options == 'Video':       
    video_file = st.file_uploader("Choose an Video...", type=["mp4", "avi", "mov"])
    if video_file is not None:
       with tempfile.NamedTemporaryFile(delete=False) as temp_video:
            temp_video.write(video_file.read())
    cap = cv2.VideoCapture(temp_video.name)
    frame_count = 0
    freme_rete = 50
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
           break
        freme_count += 1
        if frame_count % freme_rete == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            emotion_scores = Analyze_Emotion(frame_rgb)
            if emotion_scores:
                detected_emotion = max(emotion_scores, key=emotion_scores.get)
                cv2.putText(frame, f"Emotion: {detected_emotion}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            else:
                detected_emotion = "No emotion detected"
                cv2.putText(frame, detected_emotion, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    
            st.image(frame, channels="RGB", caption='Video Frame{frame_count}')    
            
        cap.release()
        cv2.destroyAllWindows()

Overwriting app.py


In [23]:
%%writefile app.py
import streamlit as st
from deepface import DeepFace
import cv2
import numpy as np
from PIL import Image
import tempfile

st.set_page_config(page_title="Human Emojis - Emotion Detection", layout="centered")

st.title("😊 Human Emojis - Emotion Detection App")
st.markdown(
    """
    <style>
    .reportview-container {
        background: #f5f7fa;
    }
    .sidebar .sidebar-content {
        background: #e3e6ec;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

def Analyze_Emotion(img):
    try:
        analysis = DeepFace.analyze(
            img,
            actions=['emotion'],
            enforce_detection=False
        )
        return analysis[0]['emotion']
    except Exception as e:
        st.error(f"Error analyzing image: {e}")
        return None

option = st.selectbox('Select an option', ('Upload Image', 'Upload Video'))

if option == 'Upload Image':
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png" , "jfif"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        img_array = np.array(image)
        st.image(img_array, caption='Uploaded Image', use_column_width=True)
        emotion_scores = Analyze_Emotion(img_array)
        if emotion_scores:
            detected_emotion = max(emotion_scores, key=emotion_scores.get)
            st.success(f"**Detected Emotion:** {detected_emotion}")
        else:
            st.warning("No emotion detected or an error occurred.")

elif option == 'Upload Video':
    video_file = st.file_uploader("Choose a video...", type=["mp4", "avi", "mov"])
    if video_file is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_video:
            temp_video.write(video_file.read())
            temp_video_path = temp_video.name
        cap = cv2.VideoCapture(temp_video_path)
        frame_count = 0
        frame_rate = 50
        stframe = st.empty()
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_count += 1
            if frame_count % frame_rate == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                emotion_scores = Analyze_Emotion(frame_rgb)
                if emotion_scores:
                    detected_emotion = max(emotion_scores, key=emotion_scores.get)
                    cv2.putText(frame_rgb, f"Emotion: {detected_emotion}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                else:
                    cv2.putText(frame_rgb, "No emotion detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                stframe.image(frame_rgb, channels="RGB", caption=f'Video Frame {frame_count}')
        cap.release()
        cv2.destroyAllWindows()

Overwriting app.py
